In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install pycaret
from pycaret.classification import *
#from pycaret.regression import *

# Importing the data - only use *.MLReady.csv files!!!

In [2]:
df=pd.read_csv("D:\FM_Acq_Perf_FMAC_EPOCH3_ML.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,...,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,1265166,1265166,100495721320,1,4.375,310000,2013-11-01,2014-01-01,64.0,1.0,...,4.16,4.255,0.045673,2.65,0.025,-0.215,0.120,0.005747,-0.051683,0.028202
1,1265167,1265167,102361386857,1,4.625,142000,2013-11-01,2014-01-01,69.0,1.0,...,4.16,4.255,0.045673,2.65,0.275,-0.465,0.370,0.063218,-0.111779,0.086957
2,1265168,1265168,102516953457,1,5.000,124000,2013-11-01,2014-01-01,80.0,1.0,...,4.16,4.255,0.045673,2.65,0.650,-0.840,0.745,0.149425,-0.201923,0.175088
3,1265169,1265169,103711862089,3,4.490,170000,2013-11-01,2014-01-01,54.0,2.0,...,4.16,4.255,0.045673,2.65,0.140,-0.330,0.235,0.032184,-0.079327,0.055229
4,1265170,1265170,104417831957,3,4.750,100000,2013-11-01,2014-01-01,32.0,1.0,...,4.16,4.255,0.045673,2.65,0.400,-0.590,0.495,0.091954,-0.141827,0.116334


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35388 entries, 0 to 35387
Data columns (total 60 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  35388 non-null  int64  
 1   Unnamed: 0.1                35388 non-null  int64  
 2   id                          35388 non-null  int64  
 3   origChannel                 35388 non-null  int64  
 4   origIntRate                 35388 non-null  float64
 5   origUPB                     35388 non-null  int64  
 6   origDate                    35388 non-null  object 
 7   firstPmtDate                35388 non-null  object 
 8   origLTV                     35388 non-null  float64
 9   numBorrowers                35388 non-null  float64
 10  origDebtIncRatio            35388 non-null  float64
 11  borrCreditScore             35388 non-null  float64
 12  loanPurp                    35388 non-null  int64  
 13  zipCode                     353

## Remove columns not involved in 'Foreclosure or Not' query

In [4]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','firstPmtDate','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
         'zeroBalDate','lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost','recovCosts',\
         'miscCost','holdTaxCost','saleProceed','credEnhProceed','repurchProceed','otherForecloProceed',\
         'nonIntUPB','prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator',\
         'deliqGood','deliqBad','deliqMax'],1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35388 entries, 0 to 35387
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         35388 non-null  int64  
 1   origIntRate         35388 non-null  float64
 2   origUPB             35388 non-null  int64  
 3   origDate            35388 non-null  object 
 4   origLTV             35388 non-null  float64
 5   numBorrowers        35388 non-null  float64
 6   origDebtIncRatio    35388 non-null  float64
 7   borrCreditScore     35388 non-null  float64
 8   loanPurp            35388 non-null  int64  
 9   zipCode             35388 non-null  int64  
 10  pMIperct            35388 non-null  float64
 11  mortInsType         35388 non-null  float64
 12  bestCreditScore     35388 non-null  float64
 13  worstCreditScore    35388 non-null  float64
 14  avgCreditScore      35388 non-null  float64
 15  bankNumber          35388 non-null  int64  
 16  stat

## Engineer originDate into Date features

In [6]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [7]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [8]:
df.drop(['origDate'],1,inplace=True)

In [9]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2013,11
1,2013,11
2,2013,11
3,2013,11
4,2013,11


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [10]:
df.zeroBalCode.unique()

array([ 1.,  9., 16.,  3.,  2.,  6., 15.])

In [11]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,16,6,2,15],1)

In [12]:
df.zeroBalCode.unique()

array([0., 1.])

In [13]:
#Checking datatypes of individual feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35388 entries, 0 to 35387
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         35388 non-null  int64  
 1   origIntRate         35388 non-null  float64
 2   origUPB             35388 non-null  int64  
 3   origLTV             35388 non-null  float64
 4   numBorrowers        35388 non-null  float64
 5   origDebtIncRatio    35388 non-null  float64
 6   borrCreditScore     35388 non-null  float64
 7   loanPurp            35388 non-null  int64  
 8   zipCode             35388 non-null  int64  
 9   pMIperct            35388 non-null  float64
 10  mortInsType         35388 non-null  float64
 11  bestCreditScore     35388 non-null  float64
 12  worstCreditScore    35388 non-null  float64
 13  avgCreditScore      35388 non-null  float64
 14  bankNumber          35388 non-null  int64  
 15  stateNumber         35388 non-null  int64  
 16  mSA 

# Review values for dates and slice (if needed) the timeframe you want

In [14]:
df.origYear.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [15]:
df['origMonth'].unique()

array([11, 12,  1,  2,  3,  5,  4,  6,  7,  9,  8, 10], dtype=int64)

# Coerce the column dtype (int in this case)

In [16]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'int'})

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35388 entries, 0 to 35387
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         35388 non-null  int64  
 1   origIntRate         35388 non-null  float64
 2   origUPB             35388 non-null  int64  
 3   origLTV             35388 non-null  int32  
 4   numBorrowers        35388 non-null  int32  
 5   origDebtIncRatio    35388 non-null  int32  
 6   borrCreditScore     35388 non-null  int32  
 7   loanPurp            35388 non-null  int64  
 8   zipCode             35388 non-null  int64  
 9   pMIperct            35388 non-null  float64
 10  mortInsType         35388 non-null  int32  
 11  bestCreditScore     35388 non-null  int32  
 12  worstCreditScore    35388 non-null  int32  
 13  avgCreditScore      35388 non-null  int32  
 14  bankNumber          35388 non-null  int64  
 15  stateNumber         35388 non-null  int64  
 16  mSA 

In [18]:
df.to_csv('D:\FM_FULL_EPOCH3_MLReady.csv')

# Review metrics last time before run (to have record of starting values)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         119696 non-null  int64  
 1   origIntRate         119696 non-null  float64
 2   origUPB             119696 non-null  int64  
 3   origLTV             119696 non-null  int32  
 4   numBorrowers        119696 non-null  int32  
 5   origDebtIncRatio    119696 non-null  int32  
 6   borrCreditScore     119696 non-null  int32  
 7   loanPurp            119696 non-null  int64  
 8   zipCode             119696 non-null  int64  
 9   pMIperct            119696 non-null  float64
 10  mortInsType         119696 non-null  int32  
 11  bestCreditScore     119696 non-null  int32  
 12  worstCreditScore    119696 non-null  int32  
 13  avgCreditScore      119696 non-null  int32  
 14  bankNumber          119696 non-null  int64  
 15  stateNumber         119696 non-nul

In [57]:
df.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,bankNumber,stateNumber,mSA,zeroBalCode,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,2,5.125,348000,87,1,50,689,2,51,25.0,1,689,689,689,80,49,0,1,5.25,5.04,5.13,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975,2009,2
1,3,4.625,195000,52,2,54,703,1,82,0.0,0,810,703,756,4,32,12100,0,5.25,5.04,5.13,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441,2009,2
2,2,4.875,342000,80,1,54,746,1,981,0.0,0,746,746,746,3,50,42660,0,5.25,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2
3,1,5.375,93000,70,1,50,780,1,496,0.0,0,780,780,780,54,23,0,1,5.25,5.04,5.13,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758,2009,2
4,1,4.875,182000,76,2,22,802,1,18,0.0,0,802,776,789,45,20,14460,0,5.25,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2


# Pycaret - Setup with categorical definition

### Use feature engineering, polynomial features, trig feats, PCA

In [58]:
# models=setup(df, target = 'rateDiffPct',
#              feature_interaction=True, feature_ratio=True,
#              polynomial_features=True, pca=True, pca_components=10,
#             numeric_features = ['origUPB','origLTV','numBorrowers','origDebtIncRatio',\
#                                'avgCreditScore'],
#             ignore_features = ['origIntRate','rank','deal'])

### Use feature engineering, polynomial features, trig feats

In [59]:
models=setup(df, target = 'zeroBalCode')

 
Setup Succesfully Completed!


,Description,Value
0,session_id,8661
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(119696, 31)"
4,Missing Values,False
5,Numeric Features,26
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


## Compare Models with blacklist exclusions

In [ ]:
model_results=compare_models(blacklist = ['tr','ransac']) #,'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

IntProgress(value=0, description='Processing: ', max=225)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:14:35
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 4 of 10
Estimator,. . . . . . . . . . . . . . . . . .,Support Vector Machine
ETC,. . . . . . . . . . . . . . . . . .,2.21 Minutes Remaining


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.2244,0.1068,0.3267,0.0851,0.2296,0.7897
1,Ridge Regression,0.2244,0.1068,0.3267,0.0850,0.2296,0.7899
2,Bayesian Ridge,0.2248,0.1069,0.3268,0.0846,0.2296,0.7911
3,Orthogonal Matching Pursuit,0.2257,0.1076,0.3279,0.0784,0.2304,0.7960
4,Elastic Net,0.2263,0.1109,0.3329,0.0503,0.2328,0.8281
5,Huber Regressor,0.2283,0.1124,0.3352,0.0369,0.2351,0.8298
6,Lasso Regression,0.2288,0.1136,0.3370,0.0267,0.2360,0.8416
7,Lasso Least Angle Regression,0.2336,0.1168,0.3417,-0.0005,0.2392,0.8650
8,Passive Aggressive Regressor,0.2337,0.1376,0.3702,-0.1784,0.2481,0.9160
9,Least Angle Regression,0.3873,0.9220,0.5938,-6.6763,0.3084,0.9083


## Create models - store in variable

In [41]:
catboost =create_model('catboost', fold = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0444,0.0033,0.0572,0.5489,0.0477,0.6271
1,0.0440,0.0032,0.0563,0.5503,0.0463,0.4859
2,0.0425,0.0030,0.0547,0.5694,0.0456,0.4834
3,0.0442,0.0032,0.0565,0.5694,0.0471,0.5806
4,0.0425,0.0030,0.0548,0.5868,0.0452,0.4655
5,0.0448,0.0033,0.0572,0.5510,0.0474,0.4967
6,0.0442,0.0032,0.0565,0.5572,0.0463,0.5265
7,0.0442,0.0032,0.0565,0.5581,0.0470,0.4374
8,0.0437,0.0031,0.0560,0.5745,0.0464,0.5589
9,0.0436,0.0031,0.0560,0.5671,0.0460,0.4049


In [40]:
catboost =create_model('catboost', fold = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0448,0.0034,0.0585,0.5319,0.0488,0.5551
1,0.0449,0.0032,0.0567,0.5472,0.0475,0.7093
2,0.0428,0.0031,0.0555,0.5808,0.0466,0.5539
3,0.0461,0.0036,0.0599,0.5127,0.0495,0.6825
4,0.0429,0.0030,0.0549,0.5785,0.0457,0.5920
5,0.0447,0.0033,0.0575,0.5483,0.0469,0.6331
6,0.0453,0.0034,0.0581,0.5427,0.0469,0.4772
7,0.0412,0.0028,0.0528,0.5737,0.0442,0.4422
8,0.0457,0.0033,0.0572,0.5201,0.0478,0.4227
9,0.0431,0.0031,0.0558,0.5676,0.0456,0.4770


### Catboost fold results (not shown - reran in same cell)
* 10   0.5?
* 20   0.5633
* 50   0.5633

In [38]:
lightgbm = create_model('lightgbm', fold=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0453,0.0035,0.0590,0.5242,0.0493,0.5556
1,0.0453,0.0033,0.0571,0.5417,0.0478,0.7061
2,0.0431,0.0031,0.0556,0.5794,0.0467,0.5243
3,0.0460,0.0036,0.0601,0.5104,0.0495,0.6603
4,0.0431,0.0030,0.0550,0.5775,0.0458,0.5854
5,0.0448,0.0033,0.0576,0.5464,0.0470,0.6326
6,0.0461,0.0035,0.0591,0.5266,0.0476,0.4634
7,0.0415,0.0029,0.0534,0.5639,0.0446,0.4331
8,0.0453,0.0033,0.0570,0.5232,0.0475,0.4218
9,0.0434,0.0032,0.0562,0.5612,0.0459,0.5077


### LightGBM fold results (not shown - reran in same cell)
    * 10   0.5?
    * 20   0.5?
    * 50   0.5585

In [39]:
xgboost = create_model('xgboost', fold = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0463,0.0036,0.0602,0.5033,0.0501,0.5694
1,0.0473,0.0035,0.0590,0.5107,0.0489,0.7141
2,0.0443,0.0032,0.0569,0.5589,0.0477,0.6138
3,0.0475,0.0038,0.0615,0.4863,0.0505,0.7718
4,0.0450,0.0033,0.0570,0.5457,0.0473,0.5958
5,0.0467,0.0035,0.0591,0.5218,0.0480,0.6780
6,0.0469,0.0036,0.0602,0.5096,0.0483,0.5419
7,0.0423,0.0030,0.0545,0.5470,0.0451,0.4843
8,0.0471,0.0035,0.0588,0.4942,0.0487,0.4569
9,0.0448,0.0033,0.0578,0.5364,0.0466,0.5207


## Tune Models

In [29]:
tuned_catboost = tune_model('catboost') # from fold=50

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0444,0.0033,0.0573,0.5472,0.0478,0.6147
1,0.0440,0.0032,0.0563,0.5500,0.0463,0.4859
2,0.0424,0.0030,0.0546,0.5704,0.0455,0.4754
3,0.0443,0.0032,0.0567,0.5665,0.0473,0.5763
4,0.0426,0.0030,0.0550,0.5829,0.0453,0.4552
5,0.0448,0.0033,0.0573,0.5491,0.0473,0.4890
6,0.0442,0.0032,0.0568,0.5535,0.0465,0.5395
7,0.0442,0.0032,0.0565,0.5588,0.0470,0.4425
8,0.0439,0.0032,0.0563,0.5706,0.0466,0.5480
9,0.0437,0.0032,0.0561,0.5655,0.0461,0.4086


* Made no difference creating the model with fold= 10 vs. fold = 50 so use fold = 10

In [31]:
tuned_lightgbm = tune_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0479,0.0037,0.0611,0.4858,0.0503,0.6673
1,0.0474,0.0036,0.0602,0.4861,0.0489,0.5574
2,0.0460,0.0035,0.0588,0.5020,0.0483,0.5115
3,0.0477,0.0037,0.0606,0.5037,0.0496,0.5962
4,0.0466,0.0035,0.0595,0.5128,0.0482,0.5162
5,0.0478,0.0037,0.0607,0.4943,0.0495,0.5219
6,0.0481,0.0037,0.0610,0.4842,0.0493,0.5313
7,0.0480,0.0037,0.0606,0.4911,0.0499,0.4410
8,0.0478,0.0037,0.0606,0.5020,0.0495,0.5754
9,0.0474,0.0037,0.0604,0.4964,0.0488,0.4457


In [32]:
tuned_xgboost = tune_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0460,0.0036,0.0597,0.5085,0.0498,0.6511
1,0.0459,0.0035,0.0588,0.5097,0.0482,0.5120
2,0.0445,0.0033,0.0574,0.5258,0.0477,0.4845
3,0.0466,0.0036,0.0599,0.5152,0.0498,0.5903
4,0.0451,0.0034,0.0580,0.5357,0.0476,0.5077
5,0.0463,0.0035,0.0593,0.5166,0.0490,0.5207
6,0.0465,0.0036,0.0596,0.5074,0.0490,0.5561
7,0.0463,0.0035,0.0591,0.5169,0.0492,0.4543
8,0.0458,0.0035,0.0590,0.5287,0.0488,0.5824
9,0.0458,0.0035,0.0588,0.5235,0.0483,0.4413


## Ensemble models

In [28]:
catboost_bagged = ensemble_model(catboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0444,0.0033,0.0573,0.5474,0.0478,0.6229
1,0.0442,0.0032,0.0565,0.5477,0.0464,0.4915
2,0.0426,0.0030,0.0548,0.5679,0.0457,0.4841
3,0.0443,0.0032,0.0566,0.5677,0.0472,0.5791
4,0.0426,0.0030,0.0548,0.5856,0.0452,0.4571
5,0.0449,0.0033,0.0573,0.5486,0.0474,0.4938
6,0.0443,0.0032,0.0566,0.5553,0.0464,0.5284
7,0.0443,0.0032,0.0567,0.5555,0.0472,0.4329
8,0.0439,0.0032,0.0562,0.5710,0.0465,0.5604
9,0.0437,0.0032,0.0562,0.5649,0.0461,0.4175


In [30]:
catboost_boosted = ensemble_model(catboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0444,0.0033,0.0573,0.5474,0.0478,0.6229
1,0.0442,0.0032,0.0565,0.5477,0.0464,0.4915
2,0.0426,0.0030,0.0548,0.5679,0.0457,0.4841
3,0.0443,0.0032,0.0566,0.5677,0.0472,0.5791
4,0.0426,0.0030,0.0548,0.5856,0.0452,0.4571
5,0.0449,0.0033,0.0573,0.5486,0.0474,0.4938
6,0.0443,0.0032,0.0566,0.5553,0.0464,0.5284
7,0.0443,0.0032,0.0567,0.5555,0.0472,0.4329
8,0.0439,0.0032,0.0562,0.5710,0.0465,0.5604
9,0.0437,0.0032,0.0562,0.5649,0.0461,0.4175


## Blend 3 chosen Models

#### First blend 'non-tuned' models

In [27]:
blend3 = blend_models(estimator_list = [catboost, lightgbm, xgboost])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0447,0.0033,0.0575,0.5440,0.0479,0.6299
1,0.0444,0.0032,0.0567,0.5443,0.0464,0.4991
2,0.0428,0.0030,0.0550,0.5643,0.0457,0.4812
3,0.0446,0.0032,0.0569,0.5628,0.0473,0.5781
4,0.0429,0.0030,0.0551,0.5814,0.0453,0.4658
5,0.0450,0.0033,0.0574,0.5476,0.0473,0.4894
6,0.0447,0.0032,0.0570,0.5497,0.0465,0.5256
7,0.0446,0.0032,0.0568,0.5539,0.0471,0.4275
8,0.0442,0.0032,0.0564,0.5689,0.0466,0.5614
9,0.0440,0.0032,0.0564,0.5607,0.0462,0.4066


#### Next blend tuned models

In [33]:
blendTuned3 = blend_models(estimator_list = [tuned_catboost, tuned_lightgbm, tuned_xgboost])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0450,0.0034,0.0580,0.5368,0.0481,0.6316
1,0.0446,0.0032,0.0570,0.5393,0.0466,0.5032
2,0.0431,0.0031,0.0554,0.5579,0.0459,0.4753
3,0.0451,0.0033,0.0576,0.5517,0.0477,0.5729
4,0.0437,0.0031,0.0561,0.5668,0.0458,0.4769
5,0.0452,0.0033,0.0577,0.5423,0.0474,0.4965
6,0.0451,0.0033,0.0577,0.5383,0.0470,0.5267
7,0.0450,0.0033,0.0573,0.5455,0.0474,0.4320
8,0.0447,0.0033,0.0572,0.5570,0.0471,0.5537
9,0.0445,0.0032,0.0570,0.5526,0.0464,0.4176


## Evaluate Models

In [ ]:
plot_model(lightgbm, plot = 'residuals')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot='vc')

In [ ]:
plot_model(lightgbm, plot = 'feature')

In [ ]:
interpret_model(lightgbm, plot='correlation', feature='avgCreditScore')

In [ ]:
interpret_model(tuned_catboost)

In [ ]:
save_model()

In [ ]:
interpret_model(estimator=logreg_model, plot='reason')

In [ ]:
logreg_model2=create_model('xgboost')

In [ ]:
plot_model(estimator=logreg_model2, plot='feature')

In [ ]:
plot_model(estimator=logreg_model2)

In [ ]:
tunned_logreg_model=tune_model('catboost')

In [ ]:
interpret_model(estimator=tunned_logreg_model,plot='summary')

In [ ]:
plot_model(estimator=tunned_logreg_model,plot='feature')

In [ ]:
interpret_model(estimator=catboost, plot='correlation')

In [ ]:
interpret_model(estimator=catboost,plot='summary')